In [ ]:
!!!
# Настройка API ключей для колаба
!!!

#  библиотеки
!pip install -q pandas numpy scikit-learn
!pip install -q phidata openai duckduckgo-search

import os
import json
import re
from datetime import datetime
from typing import Dict, List, Any, Optional, Tuple
from dataclasses import dataclass, field
from enum import Enum
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import asyncio
import warnings
warnings.filterwarnings('ignore')


# УСТАНОВКА API КЛЮЧЕЙ (ИЗМЕНИТЕ ЭТИ ЗНАЧЕНИЯ)


# УДАЛИТЕ КОММЕНТАРИИ И ВСТАВЬТЕ ВАШИ API КЛЮЧИ НИЖЕ
# os.environ['OPENAI_API_KEY'] = 'ваш-ключ-openai-здесь'

# По умолчанию (демо режим)
os.environ['OPENAI_API_KEY'] = ''


# ИМПОРТ БИБЛИОТЕК ОАОАААААА


print("Импорт библиотек...")

try:
    # Импортируем phidata с актуальной версией
    from phi.agent import Agent
    from phi.model.openai import OpenAIChat
    from phi.tools.duckduckgo import DuckDuckGo
    from phi.tools import Toolkit
    print("Библиотека phi успешно импортирована")

except ImportError as e:
    print(f"Ошибка импорта phi: {e}")
    print("Установка библиотек...")
    !pip install -q phidata openai duckduckgo-search
    from phi.agent import Agent
    from phi.model.openai import OpenAIChat
    from phi.tools.duckduckgo import DuckDuckGo
    from phi.tools import Toolkit


# КЛАССЫ ДЛЯ AGNO GUARDRAILS


class SecurityLevel(Enum):
    """Уровни безопасности для контроля доступа"""
    PUBLIC = "public"
    INTERNAL = "internal"
    RESTRICTED = "restricted"
    CONFIDENTIAL = "confidential"

class ContentCategory(Enum):
    """Категории контента для фильтрации"""
    TECHNICAL = "technical"
    SCIENTIFIC = "scientific"
    EDUCATIONAL = "educational"
    BUSINESS = "business"
    GENERAL = "general"
    SENSITIVE = "sensitive"

@dataclass
class SecurityPolicy:
    """Политика безопасности для ограничения ввода/вывода"""
    allowed_categories: List[ContentCategory]
    security_level: SecurityLevel
    max_query_length: int = 500
    max_response_length: int = 2000
    prohibited_patterns: List[str] = field(default_factory=lambda: [
        r"секретн[а-яё]*", r"конфиденциал[а-яё]*", r"взлом", r"хакер",
        r"незаконн[а-яё]*", r"противоправн[а-яё]*", r"опасн[а-яё]*",
        r"оружие", r"наркотик", r"экстремизм", r"терроризм"
    ])

    def validate_input(self, text: str) -> Tuple[bool, Optional[str]]:
        """Валидация входного текста"""
        if len(text) > self.max_query_length:
            return False, f"Превышена максимальная длина запроса ({self.max_query_length} символов)"

        for pattern in self.prohibited_patterns:
            if re.search(pattern, text, re.IGNORECASE):
                return False, f"Обнаружен запрещенный паттерн: {pattern}"

        return True, None

    def validate_output(self, text: str) -> Tuple[bool, Optional[str]]:
        """Валидация выходного текста"""
        if len(text) > self.max_response_length:
            return False, f"Превышена максимальная длина ответа ({self.max_response_length} символов)"

        for pattern in self.prohibited_patterns:
            if re.search(pattern, text, re.IGNORECASE):
                return False, f"Ответ содержит запрещенный паттерн: {pattern}"

        return True, None


# КАСТОМНЫЙ ИНСТРУМЕНТ С GUARDRAILS


class GuardedSearchTool(Toolkit):
    """Инструмент поиска с защитой от нежелательных запросов"""

    def __init__(self, security_policy: SecurityPolicy):
        super().__init__(name="guarded_search_tool")
        self.security_policy = security_policy
        self.search_history = []

    def search(self, query: str, max_results: int = 3) -> str:
        """Поиск информации с защитой"""
        print(f"[GuardedSearchTool] Проверка запроса: {query[:50]}...")

        # Валидация запроса
        is_valid, error_message = self.security_policy.validate_input(query)
        if not is_valid:
            return f"ОШИБКА БЕЗОПАСНОСТИ: {error_message}"

        # Проверка категории запроса
        query_category = self._categorize_query(query)
        if ContentCategory.SENSITIVE in self.security_policy.allowed_categories:
            pass
        elif query_category == ContentCategory.SENSITIVE:
            return "ЗАПРОС ОТКЛОНЕН: Данная категория запросов не разрешена политикой безопасности"

        # Выполнение поиска
        try:
            # Демонстрационные результаты (для работы без API)
            demo_results = [
                {
                    "title": f"Результат по запросу: {query}",
                    "snippet": f"Информация по теме '{query}'. Это пример результата поиска с использованием Guardrails системы.",
                    "url": "https://example.com/search-result"
                },
                {
                    "title": f"Дополнительные данные по теме: {query}",
                    "snippet": "В реальной системе с настроенными API ключами здесь были бы фактические результаты поиска из интернета.",
                    "url": "https://example.com/additional-info"
                },
                {
                    "title": f"Исследовательский материал: {query}",
                    "snippet": "Система использует техники промпт-инжиниринга для улучшения качества ответов и Guardrails для безопасности.",
                    "url": "https://example.com/research-material"
                }
            ]

            self.search_history.append({
                "query": query,
                "timestamp": datetime.now().isoformat(),
                "category": query_category.value,
                "results_count": len(demo_results),
                "validated": True
            })

            formatted_results = self._format_results(demo_results)

            # Валидация результатов
            is_output_valid, output_error = self.security_policy.validate_output(formatted_results)
            if not is_output_valid:
                return f"ОШИБКА ВАЛИДАЦИИ РЕЗУЛЬТАТОВ: {output_error}"

            return formatted_results

        except Exception as e:
            return f"Ошибка при выполнении поиска: {str(e)}"

    def _categorize_query(self, query: str) -> ContentCategory:
        """Категоризация запроса на основе ключевых слов"""
        query_lower = query.lower()

        technical_keywords = ["программирование", "алгоритм", "технология", "разработка", "код", "искусственный интеллект", "машинное обучение"]
        scientific_keywords = ["исследование", "наука", "эксперимент", "теория", "гипотеза", "физика", "химия", "биология"]
        educational_keywords = ["обучение", "образование", "учебник", "курс", "лекция", "методика", "преподавание"]
        business_keywords = ["бизнес", "стратегия", "рынок", "экономика", "управление", "финансы", "инвестиции"]
        sensitive_keywords = ["секрет", "конфиденциально", "пароль", "ключ", "доступ", "взлом", "уязвимость"]

        if any(keyword in query_lower for keyword in sensitive_keywords):
            return ContentCategory.SENSITIVE
        elif any(keyword in query_lower for keyword in technical_keywords):
            return ContentCategory.TECHNICAL
        elif any(keyword in query_lower for keyword in scientific_keywords):
            return ContentCategory.SCIENTIFIC
        elif any(keyword in query_lower for keyword in educational_keywords):
            return ContentCategory.EDUCATIONAL
        elif any(keyword in query_lower for keyword in business_keywords):
            return ContentCategory.BUSINESS
        else:
            return ContentCategory.GENERAL

    def _format_results(self, results: List[Dict]) -> str:
        """Форматирование результатов поиска"""
        formatted = "=" * 60 + "\n"
        formatted += "РЕЗУЛЬТАТЫ ПОИСКА С GUARDRAILS\n"
        formatted += "=" * 60 + "\n\n"

        for i, result in enumerate(results, 1):
            formatted += f"РЕЗУЛЬТАТ {i}:\n"
            formatted += f"   Заголовок: {result['title']}\n"
            formatted += f"   Описание: {result['snippet']}\n"
            formatted += f"   Ссылка: {result['url']}\n"
            formatted += "-" * 40 + "\n"

        formatted += f"\nВсего результатов: {len(results)}\n"
        formatted += "Внимание: Это демонстрационные результаты. Для реальных результатов настройте API ключи.\n"
        formatted += "=" * 60

        return formatted

    def get_search_stats(self) -> Dict[str, Any]:
        """Получение статистики поиска"""
        if not self.search_history:
            return {"total_searches": 0, "message": "История поиска пуста"}

        total_searches = len(self.search_history)
        categories = {}
        for entry in self.search_history:
            cat = entry["category"]
            categories[cat] = categories.get(cat, 0) + 1

        return {
            "total_searches": total_searches,
            "categories": categories,
            "latest_search": self.search_history[-1]["query"] if self.search_history else None,
            "latest_timestamp": self.search_history[-1]["timestamp"] if self.search_history else None
        }


# ПРОМПТ-ИНЖИНИРИНГ ТЕХНИКИ


class PromptEngineering:
    """Класс для применения техник промпт-инжиниринга"""

    @staticmethod
    def create_few_shot_prompt(task_description: str, examples: List[Tuple[str, str]], user_query: str) -> str:
        """Создание промпта с техникой Few-Shot"""
        prompt = f"""Ты - экспертная система с доступом к знаниям и инструментам поиска.

ЦЕЛЬ ЗАДАЧИ: {task_description}

ИЗУЧИ СЛЕДУЮЩИЕ ПРИМЕРЫ КОРРЕКТНЫХ ОТВЕТОВ:
"""

        for i, (question, answer) in enumerate(examples, 1):
            prompt += f"\n{'='*50}"
            prompt += f"\nПРИМЕР {i}:"
            prompt += f"\nВОПРОС ПОЛЬЗОВАТЕЛЯ: {question}"
            prompt += f"\nЭКСПЕРТНЫЙ ОТВЕТ: {answer}"

        prompt += f"\n{'='*50}"
        prompt += f"""

НА ОСНОВЕ ПРИВЕДЕННЫХ ПРИМЕРОВ, ОТВЕТЬ НА СЛЕДУЮЩИЙ ВОПРОС:

ВОПРОС: {user_query}

СТРУКТУРА ОТВЕТА:
1. Краткое определение и ключевые понятия
2. Основные принципы работы
3. Практическое применение
4. Текущие тенденции и перспективы
5. Рекомендации для дальнейшего изучения

ПРИМЕНЯЙ Few-Shot технику для генерации качественного ответа.
"""
        return prompt

    @staticmethod
    def create_chain_of_thought_prompt(question: str, domain: str = "техническая") -> str:
        """Создание промпта с техникой Chain of Thought"""
        return f"""Ты - системный аналитик и эксперт в области {domain}.

РЕШИ ЗАДАЧУ, ИСПОЛЬЗУЯ ТЕХНИКУ "ЦЕПОЧКА РАССУЖДЕНИЙ" (CHAIN OF THOUGHT):

ВОПРОС ДЛЯ АНАЛИЗА: {question}

ПОШАГОВАЯ ИНСТРУКЦИЯ ДЛЯ АНАЛИЗА:

ШАГ 1: ДЕКОМПОЗИЦИЯ ВОПРОСА
- Разбей основной вопрос на подвопросы
- Определи ключевые термины и концепции
- Выявии скрытые предположения

ШАГ 2: СБОР И АНАЛИЗ ИНФОРМАЦИИ
- Какие данные нужны для ответа?
- Какие источники информации релевантны?
- Как проверить достоверность информации?

ШАГ 3: ЛОГИЧЕСКОЕ РАССУЖДЕНИЕ
- Построй логическую цепочку от известного к неизвестному
- Примени дедуктивные и индуктивные методы
- Проверь внутреннюю согласованность выводов

ШАГ 4: СИНТЕЗ И ВАЛИДАЦИЯ
- Объедини результаты анализа
- Проверь ответ на полноту и точность
- Сформулируй окончательный вывод

ШАГ 5: ПРЕЗЕНТАЦИЯ РЕЗУЛЬТАТА
- Структурируй ответ для ясности
- Укажи уровень уверенности в выводах
- Предложи направления для дальнейшего исследования

НАЧНИ С ШАГА 1 И ПОСЛЕДОВАТЕЛЬНО ВЫПОЛНИ ВСЕ ШАГИ.
"""

    @staticmethod
    def create_role_based_prompt(role: str, task: str, constraints: List[str] = None) -> str:
        """Создание промпта с техникой Role-Playing"""
        constraints_text = ""
        if constraints:
            constraints_text = "\nОГРАНИЧЕНИЯ И ТРЕБОВАНИЯ:\n" + "\n".join(f"• {c}" for c in constraints)

        return f"""ТЫ ИСПОЛНЯЕШЬ РОЛЬ: {role}

ТВОЯ ОСНОВНАЯ ЗАДАЧА: {task}
{constraints_text}

ИНСТРУКЦИИ ДЛЯ ВЫПОЛНЕНИЯ РОЛИ:
1. Полностью погрузись в указанную роль
2. Используй профессиональную лексику, соответствующую роли
3. Применяй методы и подходы, характерные для данной роли
4. Учитывай все указанные ограничения и требования
5. Структурируй ответ в соответствии с профессиональными стандартами

ПРОФЕССИОНАЛЬНЫЕ ТРЕБОВАНИЯ:
- Ответ должен быть точным и информативным
- Используй проверенные факты и данные
- Указывай источники информации при необходимости
- Сохраняй объективность и нейтральность
- Соблюдай этические нормы профессии

НАЧНИ СВОЙ ОТВЕТ С УКАЗАНИЯ РОЛИ И КРАТКОГО ОПИСАНИЯ ПОДХОДА.
"""

    @staticmethod
    def create_comprehensive_prompt(question: str, examples: List[Tuple[str, str]], role: str = "эксперт") -> str:
        """Создание комбинированного промпта с несколькими техниками"""
        examples_text = ""
        for i, (ex_question, ex_answer) in enumerate(examples, 1):
            examples_text += f"\nПример {i}:\nВопрос: {ex_question}\nОтвет: {ex_answer}\n"

        return f"""Ты - {role} с многолетним опытом работы.

КОМБИНИРОВАННЫЙ ПОДХОД К АНАЛИЗУ:

1. FEW-SHOT ОБУЧЕНИЕ (на примерах):
{examples_text}

2. CHAIN OF THOUGHT (цепочка рассуждений):
Проанализируй вопрос через последовательность логических шагов

3. ROLE-PLAYING (исполнение роли):
Действуй строго в рамках роли {role}

ВОПРОС ДЛЯ АНАЛИЗА: {question}

СТРУКТУРА КОМПЛЕКСНОГО ОТВЕТА:
[Часть 1: Few-Shot Анализ]
- Сравни текущий вопрос с примерами
- Выяви общие паттерны и подходы

[Часть 2: Chain of Thought]
- Шаг 1: Декомпозиция вопроса
- Шаг 2: Сбор информации
- Шаг 3: Логический анализ
- Шаг 4: Синтез результатов

[Часть 3: Role-Specific Insights]
- Профессиональные инсайты как {role}
- Практические рекомендации
- Ограничения и предостережения

[Часть 4: Итоговый синтез]
- Сводка ключевых выводов
- Практическая ценность
- Дальнейшие шаги

СФОРМИРУЙ ПОЛНЫЙ ОТВЕТ, ИСПОЛЬЗУЯ ВСЕ ТРИ ТЕХНИКИ.
"""


# КАСТОМНЫЙ АГЕНТ С ПРОМПТ-ИНЖИНИРИНГОМ И GUARDRAILS


class ResearchAgent:
    """Агент для исследовательских задач с промпт-инжинирингом и защитой"""

    def __init__(
        self,
        name: str,
        role: str,
        security_policy: SecurityPolicy,
        use_real_api: bool = False
    ):
        self.name = name
        self.role = role
        self.security_policy = security_policy
        self.use_real_api = use_real_api
        self.knowledge_base = []

        # Инициализация инструментов с guardrails
        self.search_tool = GuardedSearchTool(security_policy)

        # Инициализация агента phi
        self.agent = self._initialize_agent()

        # Примеры для Few-Shot обучения
        self.few_shot_examples = self._get_domain_examples()

        print(f"Агент инициализирован: {name} ({role})")
        print(f"  Уровень безопасности: {security_policy.security_level.value}")
        print(f"  Режим API: {'РЕАЛЬНЫЙ' if use_real_api else 'ДЕМО'}")

    def _initialize_agent(self) -> Agent:
        """Инициализация агента phi с защитой"""
        tools = [self.search_tool]

        system_prompt = self._create_system_prompt()

        if self.use_real_api and os.environ.get('OPENAI_API_KEY'):
            try:
                model = OpenAIChat(model="gpt-3.5-turbo")
                print(f"  Используется модель: OpenAI GPT-3.5 Turbo")
            except:
                model = None
                print(f"  Предупреждение: Не удалось инициализировать OpenAI модель")
        else:
            model = None
            print(f"  Используется демонстрационный режим")

        return Agent(
            name=self.name,
            role=self.role,
            tools=tools,
            instructions=system_prompt,
            model=model,
            show_tool_calls=False,
            markdown=True,
            debug_mode=False
        )

    def _create_system_prompt(self) -> str:
        """Создание системного промпта с ограничениями"""
        return f"""# СИСТЕМНАЯ РОЛЬ И НАСТРОЙКИ

ТЫ: {self.name}
РОЛЬ: {self.role}
УРОВЕНЬ ДОСТУПА: {self.security_policy.security_level.value}

# ПОЛИТИКА БЕЗОПАСНОСТИ (GUARDRAILS)

## ОГРАНИЧЕНИЯ ВВОДА:
- Максимальная длина запроса: {self.security_policy.max_query_length} символов
- Разрешенные категории: {', '.join([c.value for c in self.security_policy.allowed_categories])}
- Запрещенные темы: конфиденциальная информация, вредоносный контент, незаконная деятельность

## ОГРАНИЧЕНИЯ ВЫВОДА:
- Максимальная длина ответа: {self.security_policy.max_response_length} символов
- Требуется проверка достоверности информации
- Запрещено раскрывать чувствительную информацию

## ТРЕБОВАНИЯ К КАЧЕСТВУ:
1. ТОЧНОСТЬ: Используй только проверенные факты
2. ПОЛНОТА: Давай развернутые, но лаконичные ответы
3. СТРУКТУРИРОВАННОСТЬ: Организуй информацию логически
4. ИСТОЧНИКИ: Указывай источники при использовании внешней информации
5. БЕЗОПАСНОСТЬ: Прекращай выполнение при нарушении политики безопасности

# ИНСТРУКЦИИ ДЛЯ ПОЛЬЗОВАТЕЛЯ
Система готова к работе. Задавайте вопросы в рамках установленной политики безопасности.
"""

    def _get_domain_examples(self) -> List[Tuple[str, str]]:
        """Получение примеров для Few-Shot обучения в зависимости от роли"""
        role_lower = self.role.lower()

        if "технический" in role_lower or "программист" in role_lower:
            return [
                ("Что такое объектно-ориентированное программирование?",
                 "ООП - это парадигма программирования, основанная на концепции 'объектов', которые содержат данные в виде полей и код в виде методов. Основные принципы: инкапсуляция, наследование, полиморфизм и абстракция. Примеры языков: Java, C++, Python."),
                ("Объясни разницу между HTTP и HTTPS",
                 "HTTP (HyperText Transfer Protocol) - протокол передачи данных без шифрования. HTTPS - защищенная версия HTTP с использованием SSL/TLS шифрования. Ключевые различия: безопасность (HTTPS шифрует данные), порты (HTTP: 80, HTTPS: 443), необходимость сертификата для HTTPS.")
            ]
        elif "научный" in role_lower or "исследователь" in role_lower:
            return [
                ("Что такое метод научного исследования?",
                 "Метод научного исследования - это систематический подход к получению знаний. Основные этапы: наблюдение, формулировка гипотезы, эксперимент, анализ данных, выводы. Методы включают: экспериментальный, наблюдательный, сравнительный, моделирование."),
                ("Объясни принцип работы CRISPR-Cas9",
                 "CRISPR-Cas9 - это технология редактирования генома. Принцип работы: система использует направляющую РНК для нахождения специфической последовательности ДНК и фермент Cas9 для разрезания ДНК в этом месте. Это позволяет удалять, добавлять или изменять гены.")
            ]
        elif "бизнес" in role_lower or "аналитик" in role_lower:
            return [
                ("Что такое SWOT-анализ?",
                 "SWOT-анализ - это стратегический инструмент для оценки: Strengths (сильные стороны), Weaknesses (слабые стороны), Opportunities (возможности), Threats (угрозы). Применяется для стратегического планирования и принятия решений в бизнесе."),
                ("Объясни разницу между лидерством и управлением",
                 "Лидерство - это способность влиять и вдохновлять людей для достижения целей. Управление - это процесс планирования, организации и контроля ресурсов. Лидеры задают направление, менеджеры обеспечивают выполнение. Оба навыка важны для успеха организации.")
            ]
        else:
            return [
                ("Как работает искусственный интеллект?",
                 "ИИ имитирует человеческий интеллект с помощью алгоритмов и моделей. Основные подходы: машинное обучение (обучение на данных), глубокое обучение (нейронные сети), обработка естественного языка. Применения: распознавание образов, прогнозирование, автоматизация."),
                ("Что такое блокчейн технология?",
                 "Блокчейн - это распределенная база данных, состоящая из блоков, связанных криптографически. Ключевые особенности: децентрализация, неизменяемость, прозрачность. Применения: криптовалюты, смарт-контракты, цепочки поставок.")
            ]

    async def research_topic(
        self,
        topic: str,
        use_few_shot: bool = False,
        use_chain_of_thought: bool = False,
        use_comprehensive: bool = False,
        search_enabled: bool = True
    ) -> Dict[str, Any]:
        """Исследование темы с выбранными техниками промпт-инжиниринга"""

        print(f"\n{'='*60}")
        print(f"АГЕНТ: {self.name}")
        print(f"ТЕМА: {topic}")
        print(f"УРОВЕНЬ БЕЗОПАСНОСТИ: {self.security_policy.security_level.value}")
        print(f"{'='*60}")

        # Валидация входного запроса
        is_input_valid, input_error = self.security_policy.validate_input(topic)
        if not is_input_valid:
            error_msg = f"ОШИБКА ВАЛИДАЦИИ ВХОДА: {input_error}"
            print(f"{error_msg}")
            return {
                "agent_name": self.name,
                "topic": topic,
                "error": error_msg,
                "validation_passed": False,
                "timestamp": datetime.now().isoformat()
            }

        # Выбор техники промпт-инжиниринга
        if use_comprehensive:
            prompt_technique = "comprehensive"
            prompt = PromptEngineering.create_comprehensive_prompt(
                question=topic,
                examples=self.few_shot_examples,
                role=self.role
            )
            print("ТЕХНИКА: Комплексная (Few-Shot + Chain of Thought + Role-Playing)")

        elif use_few_shot:
            prompt_technique = "few_shot"
            prompt = PromptEngineering.create_few_shot_prompt(
                task_description=f"Исследуй тему '{topic}' как {self.role}",
                examples=self.few_shot_examples,
                user_query=topic
            )
            print("ТЕХНИКА: Few-Shot Learning")

        elif use_chain_of_thought:
            prompt_technique = "chain_of_thought"
            domain = "техническая" if "технический" in self.role.lower() else "научная"
            prompt = PromptEngineering.create_chain_of_thought_prompt(topic, domain)
            print("ТЕХНИКА: Chain of Thought")

        else:
            prompt_technique = "role_based"
            constraints = [
                f"Длина ответа: не более {self.security_policy.max_response_length} символов",
                f"Разрешенные категории: {', '.join([c.value for c in self.security_policy.allowed_categories])}",
                "Требуется структурированный ответ с разделами"
            ]
            prompt = PromptEngineering.create_role_based_prompt(
                role=f"{self.name} ({self.role})",
                task=f"Исследуй тему: {topic}",
                constraints=constraints
            )
            print("ТЕХНИКА: Role-Playing")

        # Добавление инструкции по поиску если включено
        if search_enabled and self.use_real_api:
            prompt += "\n\nИНСТРУКЦИЯ ДЛЯ ПОИСКА:\nИспользуй инструмент поиска для получения актуальной информации по теме. Проверяй несколько источников для обеспечения достоверности."

        print(f"ДЛИНА ПРОМПТА: {len(prompt)} символов")
        print(f"ВЫПОЛНЕНИЕ ЗАПРОСА...")

        try:
            # Выполнение запроса
            start_time = datetime.now()

            if self.use_real_api and os.environ.get('OPENAI_API_KEY'):
                response = await self.agent.arun(prompt)
                content = response.content
            else:
                # Демонстрационный ответ
                await asyncio.sleep(2.0)
                content = self._generate_demo_response(topic, prompt_technique)

            execution_time = (datetime.now() - start_time).total_seconds()

            # Валидация вывода
            is_output_valid, output_error = self.security_policy.validate_output(content)
            if not is_output_valid:
                error_msg = f"ОШИБКА ВАЛИДАЦИИ ВЫВОДА: {output_error}"
                print(f"{error_msg}")
                content = f"{error_msg}\n\n{'='*40}\nЧАСТИЧНЫЙ ОТВЕТ (первые 500 символов):\n{content[:500]}...\n{'='*40}"

            # Сохранение в базу знаний
            self._add_to_knowledge_base(topic, content, prompt_technique)

            # Статистика поиска
            search_stats = self.search_tool.get_search_stats()

            result = {
                "agent_name": self.name,
                "agent_role": self.role,
                "topic": topic,
                "response": content,
                "prompt_technique": prompt_technique,
                "response_length": len(content),
                "execution_time_seconds": round(execution_time, 2),
                "timestamp": datetime.now().isoformat(),
                "validation_passed": is_input_valid and is_output_valid,
                "search_stats": search_stats,
                "knowledge_base_id": len(self.knowledge_base),
                "security_level": self.security_policy.security_level.value
            }

            print(f"ИССЛЕДОВАНИЕ ЗАВЕРШЕНО")
            print(f"   Время выполнения: {execution_time:.2f} сек")
            print(f"   Длина ответа: {len(content):,} символов")
            print(f"   Валидация: {'ПРОЙДЕНА' if result['validation_passed'] else 'НЕ ПРОЙДЕНА'}")
            print(f"   ID в базе знаний: {result['knowledge_base_id']}")

            return result

        except Exception as e:
            error_msg = f"ОШИБКА ВЫПОЛНЕНИЯ: {str(e)}"
            print(f"{error_msg}")
            return {
                "agent_name": self.name,
                "topic": topic,
                "error": error_msg,
                "timestamp": datetime.now().isoformat(),
                "validation_passed": False
            }

    def _generate_demo_response(self, topic: str, technique: str) -> str:
        """Генерация демонстрационного ответа с учетом техники"""

        if technique == "chain_of_thought":
            return f"""
АНАЛИЗ С ИСПОЛЬЗОВАНИЕМ CHAIN OF THOUGHT

ТЕМА АНАЛИЗА: {topic}

{'='*60}
ШАГ 1: ДЕКОМПОЗИЦИЯ ВОПРОСА
{'='*60}

Ключевые компоненты вопроса:
1. Основное понятие: {topic}
2. Связанные концепции: определение, принципы, применение
3. Контекст: современные технологии и исследования

Подвопросы для анализа:
- Что такое {topic} в базовом определении?
- Каковы основные принципы работы?
- Где и как применяется на практике?
- Каковы текущие тенденции и перспективы?

{'='*60}
ШАГ 2: СБОР И АНАЛИЗ ИНФОРМАЦИИ
{'='*60}

Источники информации:
1. Академические публикации и исследования
2. Техническая документация
3. Отраслевые отчеты и аналитика
4. Экспертные мнения и кейсы

Критерии достоверности:
- Научная обоснованность
- Практическая проверка
- Экспертное признание
- Актуальность информации

{'='*60}
ШАГ 3: ЛОГИЧЕСКОЕ РАССУЖДЕНИЕ
{'='*60}

Логическая цепочка:
1. От общего к частному: общая концепция → специфические аспекты
2. От теории к практике: принципы → применение
3. От настоящего к будущему: текущее состояние → перспективы

Дедуктивные выводы:
- Если {topic} основан на принципах X, то он должен иметь свойства Y
- Если применяется в области Z, то должен решать проблемы этой области

{'='*60}
ШАГ 4: СИНТЕЗ И ВАЛИДАЦИАЦИЯ
{'='*60}

Ключевые выводы:
1. {topic} представляет собой важную технологическую/научную концепцию
2. Основная ценность заключается в [основное преимущество]
3. Ограничения включают [основные ограничения]
4. Перспективы развития связаны с [ключевые направления]

Проверка полноты:
- Определение и базовые понятия
- Принципы работы
- Применение и кейсы
- Тенденции и перспективы

{'='*60}
ШАГ 5: ПРЕЗЕНТАЦИЯ РЕЗУЛЬТАТА
{'='*60}

ИТОГОВЫЙ ОТВЕТ:

{topic} - это [краткое определение].

ОСНОВНЫЕ ХАРАКТЕРИСТИКИ:
- [Характеристика 1]
- [Характеристика 2]
- [Характеристика 3]

ПРИНЦИПЫ РАБОТЫ:
1. [Принцип 1]
2. [Принцип 2]
3. [Принцип 3]

ПРАКТИЧЕСКОЕ ПРИМЕНЕНИЕ:
- Область 1: [описание применения]
- Область 2: [описание применения]
- Область 3: [описание применения]

ПЕРСПЕКТИВЫ РАЗВИТИЯ:
- Краткосрочные: [1-2 года]
- Среднесрочные: [3-5 лет]
- Долгосрочные: [5+ лет]

УРОВЕНЬ УВЕРЕННОСТИ В ВЫВОДАХ: 85%
ОСНОВАНИЕ: Анализ основан на стандартных методологиях исследования

РЕКОМЕНДАЦИИ ДЛЯ ДАЛЬНЕЙШЕГО ИЗУЧЕНИЯ:
1. [Рекомендуемый ресурс 1]
2. [Рекомендуемый ресурс 2]
3. [Рекомендуемый ресурс 3]

ПРИМЕЧАНИЕ: Это демонстрационный ответ, сгенерированный с использованием техники Chain of Thought.
   В реальном режиме с API ответ был бы основан на актуальных данных и исследованиях.
"""
        else:
            return f"""
{'='*60}
КОМПЛЕКСНЫЙ ОТЧЕТ ПО ТЕМЕ: {topic}
{'='*60}

АГЕНТ: {self.name} ({self.role})
ДАТА АНАЛИЗА: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}
ТЕХНИКА ПРОМПТ-ИНЖИНИРИНГА: {technique.upper()}
УРОВЕНЬ БЕЗОПАСНОСТИ: {self.security_policy.security_level.value}

{'='*60}
1. ОПРЕДЕЛЕНИЕ И КЛЮЧЕВЫЕ ПОНЯТИЯ
{'='*60}

{topic} - это [базовое определение и сущность].

КЛЮЧЕВЫЕ ТЕРМИНЫ:
- Термин 1: [определение]
- Термин 2: [определение]
- Термин 3: [определение]

ИСТОРИЧЕСКИЙ КОНТЕКСТ:
- Год возникновения/открытия: [год]
- Основные contributors: [имена]
- Эволюция концепции: [краткая история]

{'='*60}
2. ТЕХНИЧЕСКИЕ/НАУЧНЫЕ ОСНОВЫ
{'='*60}

ФУНДАМЕНТАЛЬНЫЕ ПРИНЦИПЫ:
1. [Принцип 1]: [подробное описание]
2. [Принцип 2]: [подробное описание]
3. [Принцип 3]: [подробное описание]

АРХИТЕКТУРА/СТРУКТУРА:
- Компонент A: [функция и назначение]
- Компонент B: [функция и назначение]
- Взаимодействие компонентов: [схема работы]

МАТЕМАТИЧЕСКАЯ/ТЕОРЕТИЧЕСКАЯ ОСНОВА:
- Формулы/уравнения: [примеры]
- Теоремы/законы: [применяемые законы]
- Модели: [используемые модели]

{'='*60}
3. ПРАКТИЧЕСКОЕ ПРИМЕНЕНИЕ И КЕЙСЫ
{'='*60}

ОСНОВНЫЕ ОБЛАСТИ ПРИМЕНЕНИЯ:

ПРОМЫШЛЕННОСТЬ:
- Применение 1: [описание и пример]
- Применение 2: [описание и пример]
- Экономический эффект: [оценка]

НАУКА И ИССЛЕДОВАНИЯ:
- Научное применение 1: [описание]
- Научное применение 2: [описание]
- Значение для науки: [важность]

ПОВСЕДНЕВНАЯ ЖИЗНЬ:
- Бытовое применение 1: [пример]
- Бытовое применение 2: [пример]
- Влияние на общество: [социальный аспект]

РЕАЛЬНЫЕ КЕЙСЫ:
- Кейс 1: [компания/организация] - [результат]
- Кейс 2: [компания/организация] - [результат]
- Кейс 3: [компания/организация] - [результат]

{'='*60}
4. ТЕКУЩЕЕ СОСТОЯНИЕ И ТЕНДЕНЦИИ
{'='*60}

МИРОВЫЕ ТЕНДЕНЦИИ (2024):
- Растущие направления:
  1. [Тренд 1] - темп роста: [X]%
  2. [Тренд 2] - темп роста: [Y]%
  3. [Тренд 3] - темп роста: [Z]%

- Сокращающиеся направления:
  1. [Устаревающая технология 1]
  2. [Устаревающая технология 2]

ИНВЕСТИЦИИ И РАЗВИТИЕ:
- Глобальные инвестиции: [объем в $]
- Ключевые игроки: [компании/страны]
- Инновационные стартапы: [примеры]

{'='*60}
5. ПЕРСПЕКТИВЫ И ВЫЗОВЫ
{'='*60}

ПЕРСПЕКТИВЫ НА 5-10 ЛЕТ:
- Прорывные направления:
  1. [Перспектива 1] - потенциал: [описание]
  2. [Перспектива 2] - потенциал: [описание]
  3. [Перспектива 3] - потенциал: [описание]

ОСНОВНЫЕ ВЫЗОВЫ:
- Технические: [проблемы и ограничения]
- Экономические: [финансовые барьеры]
- Регуляторные: [законодательные ограничения]
- Этические: [этические вопросы]

РОССИЙСКИЙ КОНТЕКСТ:
- Текущее состояние в РФ: [анализ]
- Перспективы развития: [прогноз]
- Государственная поддержка: [меры поддержки]

{'='*60}
6. РЕКОМЕНДАЦИИ И ВЫВОДЫ
{'='*60}

КЛЮЧЕВЫЕ ВЫВОДЫ:
1. Вывод 1: [суть вывода]
2. Вывод 2: [суть вывода]
3. Вывод 3: [суть вывода]

ПРАКТИЧЕСКИЕ РЕКОМЕНДАЦИИ:
- Для разработчиков: [рекомендации]
- Для исследователей: [рекомендации]
- Для бизнеса: [рекомендации]
- Для государства: [рекомендации]

РЕСУРСЫ ДЛЯ УГЛУБЛЕННОГО ИЗУЧЕНИЯ:
- Книги: [список]
- Курсы: [список]
- Сообщества: [список]
- Конференции: [список]

{'='*60}
ИНФОРМАЦИЯ ОБ ОТЧЕТЕ
{'='*60}

СГЕНЕРИРОВАНО: Системой исследовательских агентов с Guardrails
ТЕХНОЛОГИИ: Prompt Engineering + Agno Guardrails
РЕЖИМ: {'РЕАЛЬНЫЙ (с API)' if self.use_real_api else 'ДЕМОНСТРАЦИОННЫЙ'}
ПРОВЕРКА БЕЗОПАСНОСТИ: ПРОЙДЕНА
ДОСТОВЕРНОСТЬ: На основе общедоступных знаний и методологий

СОВЕТ: Для получения актуальных данных с реальным поиском настройте API ключи OpenAI.
"""

    def _add_to_knowledge_base(self, topic: str, content: str, technique: str):
        """Добавление результатов в базу знаний"""
        entry = {
            "id": len(self.knowledge_base) + 1,
            "topic": topic,
            "content_preview": content[:500] + "..." if len(content) > 500 else content,
            "technique": technique,
            "agent": self.name,
            "timestamp": datetime.now().isoformat(),
            "content_length": len(content),
            "security_level": self.security_policy.security_level.value
        }
        self.knowledge_base.append(entry)

    def get_knowledge_summary(self) -> Dict[str, Any]:
        """Получение статистики по базе знаний"""
        if not self.knowledge_base:
            return {
                "total_entries": 0,
                "message": "База знаний пуста",
                "agent": self.name
            }

        topics = [entry["topic"] for entry in self.knowledge_base]
        techniques = [entry["technique"] for entry in self.knowledge_base]
        total_content_length = sum(entry["content_length"] for entry in self.knowledge_base)

        # Анализ по техникам
        technique_stats = {}
        for tech in set(techniques):
            tech_entries = [e for e in self.knowledge_base if e["technique"] == tech]
            technique_stats[tech] = {
                "count": len(tech_entries),
                "avg_length": sum(e["content_length"] for e in tech_entries) // len(tech_entries)
            }

        return {
            "agent_name": self.name,
            "total_entries": len(self.knowledge_base),
            "unique_topics": len(set(topics)),
            "total_content_length": total_content_length,
            "avg_content_length": total_content_length // len(self.knowledge_base),
            "technique_statistics": technique_stats,
            "last_research": self.knowledge_base[-1]["timestamp"] if self.knowledge_base else None,
            "last_topic": self.knowledge_base[-1]["topic"] if self.knowledge_base else None,
            "security_level": self.security_policy.security_level.value
        }

    def display_knowledge_base(self, limit: int = 5):
        """Отображение базы знаний"""
        print(f"\n{'='*60}")
        print(f"БАЗА ЗНАНИЙ АГЕНТА: {self.name}")
        print(f"{'='*60}")

        if not self.knowledge_base:
            print("База знаний пуста")
            return

        entries_to_show = self.knowledge_base[-limit:] if limit > 0 else self.knowledge_base

        for i, entry in enumerate(reversed(entries_to_show), 1):
            print(f"\nЗАПИСЬ #{entry['id']}")
            print(f"   Тема: {entry['topic']}")
            print(f"   Техника: {entry['technique']}")
            print(f"   Дата: {entry['timestamp'][:19]}")
            print(f"   Длина: {entry['content_length']:,} символов")
            print(f"   Уровень безопасности: {entry['security_level']}")
            print(f"   Превью: {entry['content_preview'][:100]}...")
            print(f"   {'─'*40}")

        summary = self.get_knowledge_summary()
        print(f"\nСТАТИСТИКА:")
        print(f"   Всего записей: {summary['total_entries']}")
        print(f"   Уникальных тем: {summary['unique_topics']}")
        print(f"   Общий объем: {summary['total_content_length']:,} символов")
        print(f"   Средняя длина: {summary['avg_content_length']:,} символов")


# СИСТЕМА УПРАВЛЕНИЯ ИССЛЕДОВАТЕЛЬСКИМИ АГЕНТАМИ


class ResearchOrchestrator:
    """Оркестратор для управления несколькими исследовательскими агентами"""

    def __init__(self, use_real_api: bool = False):
        self.use_real_api = use_real_api
        self.agents = {}
        self.research_projects = []
        self.project_counter = 0

        print("\n" + "="*60)
        print("ИНИЦИАЛИЗАЦИЯ СИСТЕМЫ ИССЛЕДОВАТЕЛЬСКИХ АГЕНТОВ")
        print("="*60)

        # Инициализация агентов с различными политиками безопасности
        self._initialize_agents()

        print(f"\nСИСТЕМА ГОТОВА К РАБОТЕ")
        print(f"   Всего агентов: {len(self.agents)}")
        print(f"   Режим API: {'РЕАЛЬНЫЙ' if use_real_api else 'ДЕМОНСТРАЦИОННЫЙ'}")
        print("="*60)

    def _initialize_agents(self):
        """Инициализация агентов с разными уровнями доступа"""

        # Агент для публичных технических тем
        public_policy = SecurityPolicy(
            allowed_categories=[
                ContentCategory.TECHNICAL,
                ContentCategory.EDUCATIONAL,
                ContentCategory.GENERAL
            ],
            security_level=SecurityLevel.PUBLIC,
            max_query_length=300,
            max_response_length=1500
        )

        self.agents["tech_expert"] = ResearchAgent(
            name="Технический эксперт",
            role="Специалист по информационным технологиям и программированию",
            security_policy=public_policy,
            use_real_api=self.use_real_api
        )

        # Агент для научных исследований
        scientific_policy = SecurityPolicy(
            allowed_categories=[
                ContentCategory.SCIENTIFIC,
                ContentCategory.EDUCATIONAL,
                ContentCategory.TECHNICAL
            ],
            security_level=SecurityLevel.INTERNAL,
            max_query_length=400,
            max_response_length=2000
        )

        self.agents["science_researcher"] = ResearchAgent(
            name="Научный исследователь",
            role="Эксперт по научным методам и исследованиям",
            security_policy=scientific_policy,
            use_real_api=self.use_real_api
        )

        # Агент для бизнес-аналитики
        business_policy = SecurityPolicy(
            allowed_categories=[
                ContentCategory.BUSINESS,
                ContentCategory.GENERAL,
                ContentCategory.EDUCATIONAL
            ],
            security_level=SecurityLevel.RESTRICTED,
            max_query_length=500,
            max_response_length=2500
        )

        self.agents["business_analyst"] = ResearchAgent(
            name="Бизнес-аналитик",
            role="Специалист по бизнес-аналитике и стратегическому планированию",
            security_policy=business_policy,
            use_real_api=self.use_real_api
        )

        # Агент для образовательных тем
        education_policy = SecurityPolicy(
            allowed_categories=[
                ContentCategory.EDUCATIONAL,
                ContentCategory.GENERAL,
                ContentCategory.TECHNICAL
            ],
            security_level=SecurityLevel.PUBLIC,
            max_query_length=350,
            max_response_length=1800
        )

        self.agents["education_specialist"] = ResearchAgent(
            name="Образовательный специалист",
            role="Эксперт по методикам обучения и образовательным технологиям",
            security_policy=education_policy,
            use_real_api=self.use_real_api
        )

    async def coordinate_research_project(
        self,
        project_name: str,
        topics: List[Dict[str, Any]],
        output_format: str = "detailed"
    ) -> Dict[str, Any]:
        """Координация исследовательского проекта"""

        self.project_counter += 1
        project_id = f"PROJ-{self.project_counter:04d}-{datetime.now().strftime('%Y%m%d')}"

        print(f"\n{'='*80}")
        print(f"ЗАПУСК ИССЛЕДОВАТЕЛЬСКОГО ПРОЕКТА")
        print(f"{'='*80}")
        print(f"ID проекта: {project_id}")
        print(f"Название: {project_name}")
        print(f"Количество тем: {len(topics)}")
        print(f"Формат вывода: {output_format}")
        print(f"{'='*80}")

        project_data = {
            "project_id": project_id,
            "project_name": project_name,
            "start_time": datetime.now().isoformat(),
            "topics": topics,
            "agents_assigned": {},
            "results": {},
            "statistics": {},
            "output_format": output_format
        }

        total_successful = 0
        total_failed = 0

        for topic_idx, topic_config in enumerate(topics, 1):
            topic = topic_config.get("topic", "")
            techniques = topic_config.get("techniques", ["role_based"])
            agent_name = topic_config.get("agent", "auto")
            search_enabled = topic_config.get("search_enabled", True)

            print(f"\nТЕМА {topic_idx}/{len(topics)}")
            print(f"   Тема: {topic}")
            print(f"   Техники: {', '.join(techniques)}")
            print(f"   Агент: {agent_name}")
            print(f"   Поиск: {'ВКЛЮЧЕН' if search_enabled else 'ВЫКЛЮЧЕН'}")
            print(f"   {'─'*40}")

            # Автоматический выбор агента если не указан
            if agent_name == "auto":
                agent_name = self._select_agent_for_topic(topic)

            if agent_name not in self.agents:
                print(f"   ОШИБКА: Агент '{agent_name}' не найден")
                project_data["results"][topic] = {
                    "status": "failed",
                    "error": f"Агент '{agent_name}' не найден",
                    "assigned_agent": agent_name
                }
                total_failed += 1
                continue

            agent = self.agents[agent_name]

            # Сохраняем информацию о назначении агента
            if agent_name not in project_data["agents_assigned"]:
                project_data["agents_assigned"][agent_name] = []
            project_data["agents_assigned"][agent_name].append(topic)

            # Выполнение исследования с каждой техникой
            topic_results = {}

            for technique in techniques:
                print(f"   Выполнение с техникой: {technique}")

                try:
                    if technique == "comprehensive":
                        result = await agent.research_topic(
                            topic=topic,
                            use_comprehensive=True,
                            search_enabled=search_enabled
                        )
                    elif technique == "few_shot":
                        result = await agent.research_topic(
                            topic=topic,
                            use_few_shot=True,
                            search_enabled=search_enabled
                        )
                    elif technique == "chain_of_thought":
                        result = await agent.research_topic(
                            topic=topic,
                            use_chain_of_thought=True,
                            search_enabled=search_enabled
                        )
                    else:  # role_based по умолчанию
                        result = await agent.research_topic(
                            topic=topic,
                            search_enabled=search_enabled
                        )

                    topic_results[technique] = result

                    if "error" in result:
                        print(f"   Ошибка: {result.get('error', 'Неизвестная ошибка')}")
                    else:
                        print(f"   Успешно: {result.get('response_length', 0):,} символов")

                except Exception as e:
                    error_msg = f"Исключение при выполнении: {str(e)}"
                    print(f"   {error_msg}")
                    topic_results[technique] = {
                        "error": error_msg,
                        "status": "failed",
                        "technique": technique
                    }

            # Определяем статус темы
            successful_techniques = [k for k, v in topic_results.items()
                                   if "error" not in v and v.get("validation_passed", False)]

            if successful_techniques:
                status = "completed"
                total_successful += 1
            else:
                status = "failed"
                total_failed += 1

            project_data["results"][topic] = {
                "status": status,
                "assigned_agent": agent_name,
                "techniques_attempted": techniques,
                "techniques_successful": successful_techniques,
                "detailed_results": topic_results,
                "agent_role": agent.role,
                "security_level": agent.security_policy.security_level.value
            }

        # Завершение проекта
        project_data["completion_time"] = datetime.now().isoformat()
        project_data["statistics"] = {
            "total_topics": len(topics),
            "successful_topics": total_successful,
            "failed_topics": total_failed,
            "success_rate": (total_successful / len(topics) * 100) if topics else 0,
            "total_agents_used": len(project_data["agents_assigned"]),
            "agents_used": list(project_data["agents_assigned"].keys())
        }

        # Генерация отчета
        project_data["report"] = self._generate_project_report(project_data, output_format)

        self.research_projects.append(project_data)

        # Вывод итогов
        print(f"\n{'='*80}")
        print(f"ИТОГИ ПРОЕКТА: {project_name}")
        print(f"{'='*80}")
        print(f"Успешных тем: {total_successful}/{len(topics)}")
        print(f"Неудачных тем: {total_failed}/{len(topics)}")
        print(f"Успешность: {project_data['statistics']['success_rate']:.1f}%")
        print(f"Задействовано агентов: {project_data['statistics']['total_agents_used']}")
        print(f"Время выполнения: {project_data['start_time'][11:19]} - {project_data['completion_time'][11:19]}")
        print(f"{'='*80}")

        # Сохранение результатов
        self._save_project_results(project_data)

        return project_data

    def _select_agent_for_topic(self, topic: str) -> str:
        """Автоматический выбор агента для темы на основе ключевых слов"""
        topic_lower = topic.lower()

        # Ключевые слова для каждого агента
        tech_keywords = ["программирование", "алгоритм", "код", "разработка", "технология",
                        "искусственный интеллект", "машинное обучение", "нейронная сеть",
                        "база данных", "веб", "мобильное", "софт", "приложение"]

        science_keywords = ["исследование", "наука", "теория", "эксперимент", "методология",
                          "физика", "химия", "биология", "генетика", "квантовый", "астрономия",
                          "экология", "климат", "эволюция"]

        business_keywords = ["бизнес", "стратегия", "рынок", "экономика", "управление",
                           "финансы", "инвестиции", "стартап", "менеджмент", "маркетинг",
                           "продажи", "прибыль", "конкуренция"]

        education_keywords = ["обучение", "образование", "учебник", "курс", "лекция",
                            "методика", "преподавание", "студент", "учитель", "школа",
                            "университет", "знание", "навык"]

        # Подсчет совпадений
        tech_score = sum(1 for kw in tech_keywords if kw in topic_lower)
        science_score = sum(1 for kw in science_keywords if kw in topic_lower)
        business_score = sum(1 for kw in business_keywords if kw in topic_lower)
        education_score = sum(1 for kw in education_keywords if kw in topic_lower)

        scores = {
            "tech_expert": tech_score,
            "science_researcher": science_score,
            "business_analyst": business_score,
            "education_specialist": education_score
        }

        # Выбор агента с максимальным счетом
        selected_agent = max(scores, key=scores.get)

        # Если все счеты 0, используем технического эксперта по умолчанию
        if scores[selected_agent] == 0:
            return "tech_expert"

        return selected_agent

    def _generate_project_report(self, project_data: Dict[str, Any], format: str = "detailed") -> str:
        """Генерация отчета по проекту"""

        if format == "brief":
            report = f"ОТЧЕТ ПО ПРОЕКТУ: {project_data['project_name']}\n"
            report += f"ID: {project_data['project_id']}\n"
            report += f"Период: {project_data['start_time'][:10]} - {project_data['completion_time'][:10]}\n"
            report += f"Тем: {project_data['statistics']['total_topics']} | Успешно: {project_data['statistics']['successful_topics']}\n"
            report += f"Успешность: {project_data['statistics']['success_rate']:.1f}%\n"
            return report

        else:  # detailed format
            report_lines = [
                "=" * 80,
                f"ОТЧЕТ ИССЛЕДОВАТЕЛЬСКОГО ПРОЕКТА",
                "=" * 80,
                f"Проект: {project_data['project_name']}",
                f"Идентификатор: {project_data['project_id']}",
                f"Дата начала: {project_data['start_time']}",
                f"Дата завершения: {project_data['completion_time']}",
                "",
                "ОБЩАЯ СТАТИСТИКА:",
                f"  - Всего исследовательских тем: {project_data['statistics']['total_topics']}",
                f"  - Успешно завершено: {project_data['statistics']['successful_topics']}",
                f"  - Завершено с ошибками: {project_data['statistics']['failed_topics']}",
                f"  - Общая успешность: {project_data['statistics']['success_rate']:.1f}%",
                f"  - Задействовано агентов: {project_data['statistics']['total_agents_used']}",
                "",
                "ЗАДЕЙСТВОВАННЫЕ АГЕНТЫ:"
            ]

            for agent_name, topics in project_data['agents_assigned'].items():
                agent = self.agents[agent_name]
                report_lines.append(f"  - {agent_name} ({agent.role}): {len(topics)} тем")
                for topic in topics[:3]:  # Показываем первые 3 темы
                    report_lines.append(f"    * {topic}")
                if len(topics) > 3:
                    report_lines.append(f"    * ... и еще {len(topics) - 3} тем")

            report_lines.append("")
            report_lines.append("РЕЗУЛЬТАТЫ ПО ТЕМАМ:")
            report_lines.append("")

            for topic, data in project_data['results'].items():
                status_icon = "УСПЕХ" if data['status'] == 'completed' else "ОШИБКА"
                report_lines.append(f"{status_icon} Тема: {topic}")
                report_lines.append(f"   Агент: {data['assigned_agent']} ({data['agent_role']})")
                report_lines.append(f"   Статус: {data['status']}")
                report_lines.append(f"   Уровень безопасности: {data['security_level']}")

                if data['status'] == 'completed' and data['detailed_results']:
                    successful_techs = data['techniques_successful']
                    if successful_techs:
                        # Находим лучший результат (по длине ответа)
                        best_result = None
                        for tech, result in data['detailed_results'].items():
                            if tech in successful_techs:
                                if best_result is None or result.get('response_length', 0) > best_result.get('response_length', 0):
                                    best_result = result

                        if best_result:
                            report_lines.append(f"   Лучшая техника: {best_result['prompt_technique']}")
                            report_lines.append(f"   Длина ответа: {best_result.get('response_length', 0):,} символов")
                            report_lines.append(f"   Время выполнения: {best_result.get('execution_time_seconds', 0):.2f} сек")

                report_lines.append("")

            report_lines.append("=" * 80)
            report_lines.append("СИСТЕМНАЯ ИНФОРМАЦИЯ:")
            report_lines.append(f"  - Режим работы: {'РЕАЛЬНЫЙ API' if self.use_real_api else 'ДЕМОНСТРАЦИОННЫЙ'}")
            report_lines.append(f"  - Технологии: Prompt Engineering + Agno Guardrails")
            report_lines.append(f"  - Всего проектов в системе: {len(self.research_projects)}")
            report_lines.append("=" * 80)

            return "\n".join(report_lines)

    def _save_project_results(self, project_data: Dict[str, Any]):
        """Сохранение результатов проекта в файл"""
        filename = f"{project_data['project_id']}_{project_data['project_name'].replace(' ', '_')}.json"

        # Подготовка данных для сохранения (упрощаем для читаемости)
        save_data = {
            "project_id": project_data["project_id"],
            "project_name": project_data["project_name"],
            "timestamps": {
                "start": project_data["start_time"],
                "completion": project_data["completion_time"]
            },
            "statistics": project_data["statistics"],
            "topics_summary": [
                {
                    "topic": topic,
                    "agent": data["assigned_agent"],
                    "status": data["status"],
                    "security_level": data["security_level"]
                }
                for topic, data in project_data["results"].items()
            ],
            "report": project_data["report"]
        }

        try:
            with open(filename, 'w', encoding='utf-8') as f:
                json.dump(save_data, f, indent=2, ensure_ascii=False, default=str)

            print(f"Результаты проекта сохранены в файл: {filename}")

        except Exception as e:
            print(f"Ошибка при сохранении файла: {str(e)}")

    def get_system_statistics(self) -> Dict[str, Any]:
        """Получение статистики системы"""
        stats = {
            "system_info": {
                "total_projects": len(self.research_projects),
                "total_agents": len(self.agents),
                "use_real_api": self.use_real_api,
                "current_time": datetime.now().isoformat()
            },
            "agents_info": {},
            "knowledge_base_summary": {},
            "recent_projects": []
        }

        # Информация об агентах
        for agent_name, agent in self.agents.items():
            kb_stats = agent.get_knowledge_summary()
            stats["agents_info"][agent_name] = {
                "role": agent.role,
                "security_level": agent.security_policy.security_level.value,
                "knowledge_base_entries": kb_stats.get("total_entries", 0),
                "total_content_length": kb_stats.get("total_content_length", 0)
            }
            stats["knowledge_base_summary"][agent_name] = kb_stats

        # Недавние проекты
        recent_projects = self.research_projects[-5:] if self.research_projects else []
        for project in recent_projects:
            stats["recent_projects"].append({
                "id": project.get("project_id"),
                "name": project.get("project_name"),
                "topics": project.get("statistics", {}).get("total_topics", 0),
                "success_rate": project.get("statistics", {}).get("success_rate", 0),
                "date": project.get("completion_time", "")[:10]
            })

        return stats

    def display_system_dashboard(self):
        """Отображение дашборда системы"""
        print(f"\n{'='*80}")
        print(f"ДАШБОРД СИСТЕМЫ ИССЛЕДОВАТЕЛЬСКИХ АГЕНТОВ")
        print(f"{'='*80}")

        stats = self.get_system_statistics()

        print(f"\nОБЩАЯ СТАТИСТИКА:")
        print(f"   - Всего проектов: {stats['system_info']['total_projects']}")
        print(f"   - Зарегистрированных агентов: {stats['system_info']['total_agents']}")
        print(f"   - Режим работы: {'РЕАЛЬНЫЙ API' if stats['system_info']['use_real_api'] else 'ДЕМОНСТРАЦИОННЫЙ'}")
        print(f"   - Текущее время: {stats['system_info']['current_time'][:19]}")

        print(f"\nИНФОРМАЦИЯ ОБ АГЕНТАХ:")
        for agent_name, info in stats['agents_info'].items():
            print(f"   {agent_name}:")
            print(f"      Роль: {info['role']}")
            print(f"      Уровень безопасности: {info['security_level']}")
            print(f"      Записей в БЗ: {info['knowledge_base_entries']}")
            print(f"      Объем контента: {info['total_content_length']:,} символов")

        if stats['recent_projects']:
            print(f"\nПОСЛЕДНИЕ ПРОЕКТЫ:")
            for project in stats['recent_projects']:
                print(f"   {project['id']}: {project['name']}")
                print(f"      Тем: {project['topics']} | Успешность: {project['success_rate']:.1f}%")
                print(f"      Дата: {project['date']}")

        print(f"\nДОСТУПНЫЕ КОМАНДЫ:")
        print(f"   1. Запустить исследовательский проект")
        print(f"   2. Просмотреть базу знаний агента")
        print(f"   3. Показать статистику системы")
        print(f"   4. Сравнить техники промпт-инжиниринга")
        print(f"   5. Выход")

        print(f"\n{'='*80}")


# ТЕСТОВЫЕ ФУНКЦИИ И ДЕМОНСТРАЦИЯ


async def run_demo_project():
    """Запуск демонстрационного проекта"""

    print("\n" + "="*80)
    print("ЗАПУСК ДЕМОНСТРАЦИОННОГО ПРОЕКТА")
    print("="*80)

    # Проверка API ключей
    has_openai_key = bool(os.environ.get('OPENAI_API_KEY', '').strip())

    use_real_api = has_openai_key

    if use_real_api:
        print("РЕЖИМ: РЕАЛЬНЫЙ (обнаружен API ключ OpenAI)")
        print("   - OpenAI API: НАЙДЕН")
    else:
        print("РЕЖИМ: ДЕМОНСТРАЦИОННЫЙ (API ключ не найден)")
        print("   Для использования реального режима установите API ключ:")
        print("   os.environ['OPENAI_API_KEY'] = 'ваш-ключ'")

    print("="*80)

    # Создание оркестратора
    orchestrator = ResearchOrchestrator(use_real_api=use_real_api)

    # Определение тем для демонстрационного проекта
    demo_topics = [
        {
            "topic": "Машинное обучение и искусственный интеллект",
            "techniques": ["few_shot", "chain_of_thought"],
            "agent": "tech_expert",
            "search_enabled": True
        },
        {
            "topic": "Квантовые вычисления и их практическое применение",
            "techniques": ["chain_of_thought"],
            "agent": "science_researcher",
            "search_enabled": True
        },
        {
            "topic": "Блокчейн технологии в современной финансовой системе",
            "techniques": ["role_based", "comprehensive"],
            "agent": "business_analyst",
            "search_enabled": False
        },
        {
            "topic": "Методы онлайн-обучения и цифровые образовательные платформы",
            "techniques": ["few_shot"],
            "agent": "education_specialist",
            "search_enabled": True
        }
    ]

    # Запуск проекта
    project_results = await orchestrator.coordinate_research_project(
        project_name="Демонстрационный проект: Современные технологии",
        topics=demo_topics,
        output_format="detailed"
    )

    # Отображение дашборда
    orchestrator.display_system_dashboard()

    return {
        "orchestrator": orchestrator,
        "project_results": project_results,
        "use_real_api": use_real_api
    }

async def compare_prompt_techniques_demo():
    """Демонстрация сравнения техник промпт-инжиниринга"""

    print("\n" + "="*80)
    print("СРАВНЕНИЕ ТЕХНИК ПРОМПТ-ИНЖИНИРИНГА")
    print("="*80)

    # Создание тестового агента
    test_policy = SecurityPolicy(
        allowed_categories=[ContentCategory.TECHNICAL, ContentCategory.EDUCATIONAL],
        security_level=SecurityLevel.PUBLIC,
        max_query_length=300,
        max_response_length=1500
    )

    test_agent = ResearchAgent(
        name="Тестовый агент для сравнения",
        role="Эксперт по компьютерным наукам",
        security_policy=test_policy,
        use_real_api=False
    )

    # Тестовая тема
    test_topic = "Объясни принцип работы рекурсии в программировании"

    # Техники для сравнения
    techniques = [
        ("Без специальных техник (базовый)", {}),
        ("Few-Shot Learning", {"use_few_shot": True}),
        ("Chain of Thought", {"use_chain_of_thought": True}),
        ("Комбинированный подход", {"use_comprehensive": True})
    ]

    comparison_results = []

    print(f"\nТЕСТОВАЯ ТЕМА: {test_topic}")
    print(f"ВЫПОЛНЯЕТСЯ СРАВНЕНИЕ {len(techniques)} ТЕХНИК...")
    print("-" * 80)

    for technique_name, params in techniques:
        print(f"\nТестирование: {technique_name}")

        result = await test_agent.research_topic(
            topic=test_topic,
            use_few_shot=params.get("use_few_shot", False),
            use_chain_of_thought=params.get("use_chain_of_thought", False),
            use_comprehensive=params.get("use_comprehensive", False),
            search_enabled=False
        )

        if 'response' in result:
            response_text = result['response']
            response_length = len(response_text)

            # Анализ структуры ответа
            sections_count = response_text.count('\n#') + response_text.count('\n##') + response_text.count('\n###')
            bullet_points = response_text.count('\n•') + response_text.count('\n-') + response_text.count('\n*')
            numbered_lists = len(re.findall(r'\n\d+\.', response_text))

            # Оценка качества (упрощенная)
            has_definition = any(word in response_text.lower() for word in ["определение", "это", "означает"])
            has_examples = "пример" in response_text.lower()
            has_structure = sections_count >= 2

            quality_score = sum([has_definition, has_examples, has_structure])

            comparison_results.append({
                "technique": technique_name,
                "response_length": response_length,
                "sections": sections_count,
                "bullet_points": bullet_points,
                "numbered_lists": numbered_lists,
                "has_definition": has_definition,
                "has_examples": has_examples,
                "has_structure": has_structure,
                "quality_score": quality_score,
                "execution_time": result.get('execution_time_seconds', 0),
                "validation_passed": result.get('validation_passed', False)
            })

            print(f"   Завершено: {response_length:,} символов | Время: {result.get('execution_time_seconds', 0):.2f} сек")
            print(f"   Качество: {quality_score}/3 | Структура: {sections_count} разделов")
        else:
            print(f"   Ошибка: {result.get('error', 'Неизвестная ошибка')}")
            comparison_results.append({
                "technique": technique_name,
                "error": result.get('error', 'Неизвестная ошибка')
            })

    # Создание таблицы сравнения
    print(f"\n{'='*80}")
    print("РЕЗУЛЬТАТЫ СРАВНЕНИЯ ТЕХНИК")
    print("="*80)

    # Фильтруем успешные результаты
    successful_results = [r for r in comparison_results if 'error' not in r]

    if successful_results:
        # Создаем DataFrame для отображения
        df_data = []
        for result in successful_results:
            df_data.append({
                "Техника": result["technique"],
                "Длина": f"{result['response_length']:,}",
                "Время (сек)": f"{result['execution_time']:.2f}",
                "Разделы": result["sections"],
                "Списки": result["bullet_points"] + result["numbered_lists"],
                "Качество": f"{result['quality_score']}/3",
                "Валидация": "ДА" if result["validation_passed"] else "НЕТ"
            })

        df = pd.DataFrame(df_data)
        print("\n" + df.to_string(index=False))

        # Анализ результатов
        print(f"\n{'='*80}")
        print("АНАЛИЗ РЕЗУЛЬТАТОВ:")

        # Находим лучшую технику по каждому параметру
        if successful_results:
            best_by_length = max(successful_results, key=lambda x: x["response_length"])
            best_by_quality = max(successful_results, key=lambda x: x["quality_score"])
            best_by_speed = min(successful_results, key=lambda x: x["execution_time"])

            print(f"   Самая подробная: {best_by_length['technique']} ({best_by_length['response_length']:,} символов)")
            print(f"   Лучшее качество: {best_by_quality['technique']} ({best_by_quality['quality_score']}/3 баллов)")
            print(f"   Самая быстрая: {best_by_speed['technique']} ({best_by_speed['execution_time']:.2f} сек)")

            # Общая рекомендация
            avg_scores = {}
            for result in successful_results:
                normalized_length = result["response_length"] / max(r["response_length"] for r in successful_results)
                normalized_quality = result["quality_score"] / 3
                normalized_speed = 1 - (result["execution_time"] / max(r["execution_time"] for r in successful_results))

                # Веса: качество 40%, длина 30%, скорость 30%
                total_score = (normalized_quality * 0.4) + (normalized_length * 0.3) + (normalized_speed * 0.3)
                avg_scores[result["technique"]] = total_score

            best_overall = max(avg_scores, key=avg_scores.get)
            print(f"   Рекомендация: {best_overall} (наиболее сбалансированная)")

    return comparison_results

async def interactive_demo():
    """Интерактивная демонстрация системы"""

    print("\n" + "="*80)
    print("ИНТЕРАКТИВНАЯ ДЕМОНСТРАЦИЯ СИСТЕМЫ")
    print("="*80)

    # Проверка API ключей
    has_openai_key = bool(os.environ.get('OPENAI_API_KEY', '').strip())
    use_real_api = has_openai_key

    # Создание оркестратора
    orchestrator = ResearchOrchestrator(use_real_api=use_real_api)

    while True:
        orchestrator.display_system_dashboard()

        try:
            choice = input("\nВыберите действие (1-5): ").strip()

            if choice == "1":
                # Запуск исследовательского проекта
                print("\n" + "="*80)
                print("СОЗДАНИЕ НОВОГО ИССЛЕДОВАТЕЛЬСКОГО ПРОЕКТА")
                print("="*80)

                project_name = input("Название проекта: ").strip() or "Мой исследовательский проект"

                topics = []
                print("\nДобавьте темы для исследования (введите 'готово' для завершения):")

                topic_count = 0
                while True:
                    topic_count += 1
                    topic = input(f"\nТема {topic_count}: ").strip()

                    if topic.lower() in ['готово', 'done', ''] and topic_count > 1:
                        break
                    elif topic.lower() in ['готово', 'done', '']:
                        print("Добавьте хотя бы одну тему")
                        continue

                    print("Доступные техники: basic, few_shot, chain_of_thought, comprehensive")
                    techniques_input = input("Техники (через запятую, или Enter для basic): ").strip()
                    techniques = [t.strip() for t in techniques_input.split(',')] if techniques_input else ["basic"]

                    print("Доступные агенты: auto, tech_expert, science_researcher, business_analyst, education_specialist")
                    agent = input("Агент (или Enter для auto): ").strip() or "auto"

                    search_enabled = input("Включить поиск? (y/n, Enter для y): ").strip().lower() in ['y', 'yes', '', 'да']

                    topics.append({
                        "topic": topic,
                        "techniques": techniques,
                        "agent": agent,
                        "search_enabled": search_enabled
                    })

                    print(f"Тема '{topic}' добавлена")

                if topics:
                    await orchestrator.coordinate_research_project(
                        project_name=project_name,
                        topics=topics,
                        output_format="detailed"
                    )
                else:
                    print("Не добавлено ни одной темы")

            elif choice == "2":
                # Просмотр базы знаний
                print("\n" + "="*80)
                print("ПРОСМОТР БАЗЫ ЗНАНИЙ АГЕНТОВ")
                print("="*80)

                print("Доступные агенты:", ", ".join(orchestrator.agents.keys()))
                agent_name = input("Выберите агента: ").strip()

                if agent_name in orchestrator.agents:
                    limit_input = input("Количество записей (Enter для 5): ").strip()
                    limit = int(limit_input) if limit_input.isdigit() else 5
                    orchestrator.agents[agent_name].display_knowledge_base(limit)
                else:
                    print(f"Агент '{agent_name}' не найден")

            elif choice == "3":
                # Статистика системы
                print("\n" + "="*80)
                print("СТАТИСТИКА СИСТЕМЫ")
                print("="*80)

                stats = orchestrator.get_system_statistics()

                print(f"\nВсего проектов: {stats['system_info']['total_projects']}")
                print(f"Режим работы: {'РЕАЛЬНЫЙ API' if stats['system_info']['use_real_api'] else 'ДЕМОНСТРАЦИОННЫЙ'}")

                print(f"\nБАЗЫ ЗНАНИЙ АГЕНТОВ:")
                for agent_name, kb_stats in stats['knowledge_base_summary'].items():
                    if isinstance(kb_stats, dict) and 'total_entries' in kb_stats:
                        print(f"  {agent_name}: {kb_stats['total_entries']} записей, {kb_stats.get('total_content_length', 0):,} символов")

            elif choice == "4":
                # Сравнение техник
                print("\nЗапуск сравнения техник промпт-инжиниринга...")
                await compare_prompt_techniques_demo()

            elif choice == "5":
                print("\nЗавершение работы...")
                break

            else:
                print("Неверный выбор. Пожалуйста, выберите 1-5")

        except KeyboardInterrupt:
            print("\n\nЗавершение работы...")
            break
        except Exception as e:
            print(f"\nОшибка: {str(e)}")

    return orchestrator


# ОСНОВНАЯ ФУНКЦИЯ ЗАПУСКА


async def main():
    """Основная функция запуска системы"""

    print("\n" + "="*80)
    print("ЗАПУСК СИСТЕМЫ ИССЛЕДОВАТЕЛЬСКИХ АГЕНТОВ")
    print("С ПРОМПТ-ИНЖИНИРИНГОМ И AGNO GUARDRAILS")
    print("="*80)

    print("\nДОСТУПНЫЕ РЕЖИМЫ РАБОТЫ:")
    print("1. Автоматическая демонстрация (рекомендуется для первого запуска)")
    print("2. Интерактивный режим")
    print("3. Быстрый тест системы")

    try:
        mode = input("\nВыберите режим работы (1-3, Enter для автоматической демонстрации): ").strip()

        if mode == "2":
            print("\nЗапуск интерактивного режима...")
            orchestrator = await interactive_demo()
            return {"mode": "interactive", "orchestrator": orchestrator}

        elif mode == "3":
            print("\nЗапуск быстрого теста...")
            # Быстрый тест с одной темой
            test_policy = SecurityPolicy(
                allowed_categories=[ContentCategory.TECHNICAL],
                security_level=SecurityLevel.PUBLIC,
                max_query_length=200,
                max_response_length=1000
            )

            test_agent = ResearchAgent(
                name="Тестовый агент",
                role="Технический специалист",
                security_policy=test_policy,
                use_real_api=False
            )

            result = await test_agent.research_topic(
                topic="Что такое алгоритм?",
                use_few_shot=True
            )

            print("\n" + "="*80)
            print("БЫСТРЫЙ ТЕСТ ЗАВЕРШЕН")
            print("="*80)
            print(f"Тема: {result.get('topic')}")
            print(f"Статус: {'УСПЕХ' if 'error' not in result else 'ОШИБКА'}")
            print(f"Длина ответа: {result.get('response_length', 0):,} символов")
            print(f"Техника: {result.get('prompt_technique', 'N/A')}")

            return {"mode": "quick_test", "result": result}

        else:
            print("\nЗапуск автоматической демонстрации...")
            results = await run_demo_project()

            # Дополнительное сравнение техник
            print("\n" + "="*80)
            compare_choice = input("Выполнить сравнение техник промпт-инжиниринга? (y/n): ").strip().lower()
            if compare_choice in ['y', 'yes', 'да']:
                await compare_prompt_techniques_demo()

            return {"mode": "demo", "results": results}

    except KeyboardInterrupt:
        print("\n\nЗавершение работы по запросу пользователя")
        return {"mode": "interrupted"}
    except Exception as e:
        print(f"\nКРИТИЧЕСКАЯ ОШИБКА: {str(e)}")
        import traceback
        traceback.print_exc()
        return {"mode": "error", "error": str(e)}


# ЗАПУСК СИСТЕМЫ В GOOGLE COLAB


if __name__ == "__main__":
    print("="*80)
    print("ПОДГОТОВКА СРЕДЫ GOOGLE COLAB")
    print("="*80)

    # Проверяем, запущено ли в Google Colab
    try:
        import google.colab
        IN_COLAB = True
        print("Обнаружен Google Colab")
    except:
        IN_COLAB = False
        print("Не Google Colab - некоторые функции могут быть ограничены")

    # Информация о системе
    print(f"\nУСТАНОВЛЕННЫЕ БИБЛИОТЕКИ:")
    print(f"   - pandas: {pd.__version__}")
    print(f"   - numpy: {np.__version__}")

    print(f"\nСОСТОЯНИЕ API КЛЮЧЕЙ:")
    has_openai = bool(os.environ.get('OPENAI_API_KEY', '').strip())
    print(f"   - OpenAI API: {'НАЙДЕН' if has_openai else 'ОТСУТСТВУЕТ'}")

    if not has_openai:
        print(f"\nВНИМАНИЕ: API ключи не обнаружены!")
        print(f"   Система будет работать в ДЕМОНСТРАЦИОННОМ режиме.")
        print(f"   Для использования реального режима:")
        print(f"   1. Получите API ключ на platform.openai.com")
        print(f"   2. Вставьте его в начале кода (строка 24)")
        print(f"   3. Перезапустите среду выполнения")

    print(f"\n{'='*80}")
    print(f"НАЧАЛО РАБОТЫ СИСТЕМЫ")
    print(f"{'='*80}")

    # Запуск асинхронной основной функции
    try:
        import nest_asyncio
        nest_asyncio.apply()
        print("Применен nest_asyncio для работы в Colab")
    except:
        print("nest_asyncio не установлен, продолжаем без него")

    # Запуск основной функции
    try:
        final_results = asyncio.run(main())

        # Вывод итоговой информации
        print(f"\n{'='*80}")
        print(f"ВЫПОЛНЕНИЕ ЗАВЕРШЕНО")
        print(f"{'='*80}")

        if isinstance(final_results, dict):
            mode = final_results.get("mode", "unknown")
            print(f"Режим выполнения: {mode}")

            if mode == "demo" and "results" in final_results:
                results = final_results["results"]
                if "project_results" in results:
                    stats = results["project_results"].get("statistics", {})
                    print(f"Результаты демонстрации:")
                    print(f"  - Тем исследовано: {stats.get('total_topics', 0)}")
                    print(f"  - Успешность: {stats.get('success_rate', 0):.1f}%")
                    print(f"  - Режим API: {'РЕАЛЬНЫЙ' if results.get('use_real_api') else 'ДЕМО'}")

            print(f"\nСОЗДАННЫЕ ФАЙЛЫ:")
            import glob
            json_files = glob.glob("PROJ-*.json")
            if json_files:
                for file in sorted(json_files)[-3:]:
                    print(f"  - {file}")
            else:
                print(f"  (файлы не создавались)")

        print(f"\nСИСТЕМА УСПЕШНО ЗАВЕРШИЛА РАБОТУ")
        print(f"{'='*80}")

        # Дополнительная информация для Colab
        if IN_COLAB:
            print(f"\nСОВЕТЫ ДЛЯ GOOGLE COLAB:")
            print(f"1. Для скачивания файлов используйте: files.download('имя_файла.json')")
            print(f"2. Для перезапуска среды: Runtime -> Restart runtime")

    except KeyboardInterrupt:
        print(f"\n\nВЫПОЛНЕНИЕ ПРЕРВАНО ПОЛЬЗОВАТЕЛЕМ")
    except Exception as e:
        print(f"\nНЕПРЕДВИДЕННАЯ ОШИБКА: {str(e)}")
        import traceback
        traceback.print_exc()

    print(f"\n{'='*80}")
    print(f"СИСТЕМА ЗАВЕРШИЛА РАБОТУ")
    print(f"{'='*80}")

Импорт библиотек...
Библиотека phi успешно импортирована
ПОДГОТОВКА СРЕДЫ GOOGLE COLAB
Обнаружен Google Colab

УСТАНОВЛЕННЫЕ БИБЛИОТЕКИ:
   - pandas: 2.2.2
   - numpy: 2.0.2

СОСТОЯНИЕ API КЛЮЧЕЙ:
   - OpenAI API: ОТСУТСТВУЕТ

ВНИМАНИЕ: API ключи не обнаружены!
   Система будет работать в ДЕМОНСТРАЦИОННОМ режиме.
   Для использования реального режима:
   1. Получите API ключ на platform.openai.com
   2. Вставьте его в начале кода (строка 24)
   3. Перезапустите среду выполнения

НАЧАЛО РАБОТЫ СИСТЕМЫ
Применен nest_asyncio для работы в Colab

ЗАПУСК СИСТЕМЫ ИССЛЕДОВАТЕЛЬСКИХ АГЕНТОВ
С ПРОМПТ-ИНЖИНИРИНГОМ И AGNO GUARDRAILS

ДОСТУПНЫЕ РЕЖИМЫ РАБОТЫ:
1. Автоматическая демонстрация (рекомендуется для первого запуска)
2. Интерактивный режим
3. Быстрый тест системы

Выберите режим работы (1-3, Enter для автоматической демонстрации): 3

Запуск быстрого теста...
  Используется демонстрационный режим
Агент инициализирован: Тестовый агент (Технический специалист)
  Уровень безопасности: public
  